In [0]:
import numpy as np
import time

def mintime2d(start, end, obs):
  '''method 1: convert the 2D coordinate to ID array then solve fro min time'''
  obs.extend([start,end])
  data = np.array(obs)
  x_max = np.max(data[:,0]) + 1
  x_min = np.min(data[:,0]) - 1
  y_max = np.max(data[:,1]) + 1
  y_min = np.min(data[:,1]) - 1
  
  n = x_max - x_min
  m = y_max - y_min
  
  array_1d = []
  for [i,j] in data:
    array_1d.append(i-x_min+(y_max-j)*n)
  
  origin = array_1d[-2]
  dest = array_1d[-1]
  array_1d.pop(-1)
  array_1d.pop(-1)
  
  T = construct(m,n,origin,dest,array_1d)
  min_time = T[origin]
  u = [origin]
  while origin != dest:
    policy = [-n,n]      
    if np.mod(origin,n)==0:
      policy.append(1)
    elif np.mod(origin,n)== n-1:
      policy.append(-1)
    else: 
      policy.extend([-1,1])
    for step in policy:
      new_state = origin + step
      if (0 <= new_state < m*n): #in our domain
        if T[new_state] == T[origin] - 1: #optimal strategy path
          u.append(new_state)
          origin = new_state
          break
  U = []        
  for number in u:
    x = x_min + np.mod(number,n)
    y = y_max - int(np.floor(number/n))
    U.append([x,y])
    
  T = np.reshape(T,(m,n))  
  
  return T,U,min_time


  

def construct(m=10, n=10, origin=0, dest=0, obs=[]):
  '''Function that return the min time matrix of states'''
  T = np.zeros(m*n,dtype=int) - 2
  T[dest] = 0 
  T[obs] = -1
  bank = [dest]
  counter = 0
  if T[origin] == -1:
    raise Exception('origin is set as obstacles!')
  if T[dest] == -1:
    raise Exception('destination is set as obstacles!')
    
  while any(i==-2 for i in T):
    bank_temp = [] #to fill the new_state
    for state in bank:
      policy = [-n,n]      
      if np.mod(state,n)==0:
        policy.append(1) #only allow to move right 
      elif np.mod(state,n)== n-1:
        policy.append(-1) #only allow to move right 
      else: 
        policy.extend([-1,1])
      for step in policy:
        new_state = state + step
        if (0 <= new_state < m*n): #in our domain
          if T[new_state] == -2:
            T[new_state] = T[state]+1
            bank_temp.append(new_state)
            if T[origin] != -2:
              print('min time is found and it is ',T[origin])
              return T
                         
    bank = bank_temp
    if bank == []:
      print('some of the states have infinity minimum time, represented by -2')
      raise Exception('origin cannot reach destination at finite time')
      return T
        
        
  return T

#m = 20
#n = 10
#obs = np.random.choice(np.arange(m*n),10)
#print(obs)
#Q = construct(m,n,8,26)
#np.reshape(Q,(m,n))

np.random.seed(1000)

start_time = time.time()
m = -100
n = 100
x = np.random.randint(m,n,size=(20000,2))
print('random obstacles coordinates: ',x,'\n')
x = list(x)
T,U,t = mintime2d([62,89],[-59,-52],x)
print('min time matrix: ',T, '\n')
print('optimum path: ',U,'\n')
print('min time from origin to destination: ',t,'\n')
end_time = time.time()
print('total time used: ',end_time-start_time)




random obstacles coordinates:  [[ 79 -13]
 [-29  92]
 [ -6  -8]
 ...
 [-29 -66]
 [ 11  52]
 [-78 -54]] 

min time is found and it is  404
min time matrix:  [[230 231 232 ...  -2  -2  -2]
 [229 230 231 ...  -1  -2  -2]
 [228 229 230 ...  -2  -1  -2]
 ...
 [ 98  99  -1 ...  -2  -1  -1]
 [ 99 100  -1 ...  -2  -1  -1]
 [100 101 102 ...  -1  -2  -1]] 

optimum path:  [[62, 89], [62, 90], [62, 91], [62, 92], [61, 92], [60, 92], [60, 93], [60, 94], [59, 94], [59, 95], [59, 96], [59, 97], [59, 98], [59, 99], [59, 100], [58, 100], [57, 100], [56, 100], [55, 100], [54, 100], [53, 100], [52, 100], [51, 100], [50, 100], [49, 100], [48, 100], [47, 100], [46, 100], [45, 100], [44, 100], [43, 100], [42, 100], [41, 100], [40, 100], [39, 100], [38, 100], [37, 100], [36, 100], [35, 100], [34, 100], [33, 100], [32, 100], [31, 100], [30, 100], [29, 100], [28, 100], [27, 100], [26, 100], [25, 100], [24, 100], [23, 100], [22, 100], [21, 100], [20, 100], [19, 100], [18, 100], [17, 100], [16, 100], [15, 100],

In [0]:

import numpy as np
import time

def mintime2d(start, end, obs):
  '''method 2: utilize 2D structure then solve for min time which help to generalize
  to higher dimension'''
  obs.extend([start,end])
  data = np.array(obs)
  x_max = np.max(data[:,0]) + 1
  x_min = np.min(data[:,0]) - 1
  y_max = np.max(data[:,1]) + 1
  y_min = np.min(data[:,1]) - 1
  
  n = x_max - x_min
  m = y_max - y_min
  
  obs_norm = []
  for [i,j] in data: #since x-coordinate is actually column and y-coor is row
    obs_norm.append([(y_max-j),i-x_min])
  
  origin = [obs_norm[-2][0],obs_norm[-2][1]]
  dest = [obs_norm[-1][0],obs_norm[-1][1]]
  obs_norm.remove(origin)
  obs_norm.remove(dest)

  T = construct(m,n,np.array(origin),np.array(dest),np.array(obs_norm))
  x_origin = origin[0]
  y_origin = origin[1]
  min_time = T[x_origin,y_origin]
  u = [[x_origin,y_origin]]
  while T[x_origin,y_origin] != 0:
    for step in [1,-1]:
      x_next = x_origin + step
      if (0 <=  x_next < m): #in our domain
        if T[x_next,y_origin] == T[x_origin,y_origin] - 1:
          u.append([x_next,y_origin])
          x_origin = x_next
          break
      y_next = y_origin + step
      if (0 <=  y_next < n): #in our domain
        if T[x_origin,y_next] == T[x_origin,y_origin] - 1:        
          u.append([x_origin,y_next])
          y_origin = y_next
          break
  U = []        
  for [i,j] in u:
    x = x_min + j
    y = y_max - i
    U.append([x,y])
    
  return T,U,min_time


  

def construct(m=10, n=10, origin=[0,0], dest=[0,0], obs=[]):
  '''Function that return the min time matrix of states'''
  T = np.zeros((m,n),dtype=int) - 2
  T[dest[0],dest[1]] = 0 
  for [i,j] in obs:
    T[i,j] = -1
  bank = [dest]
  counter = 0
  if T[origin[0],origin[1]] == -1 :
    raise Exception('origin is set as obstacles!')
  if T[dest[0],dest[1]] == -1 :
    raise Exception('destination is set as obstacles!')
  
  bools = True
  while bools:
    bank_temp = [] #to fill the new_state
    for [i,j] in bank:
      for step in [1,-1]:
        i_next = i + step
        if (0 <=  i_next < m): #in our domain
          if T[i_next,j] == -2:
            T[i_next,j] = T[i,j]+1
            bank_temp.append([i_next,j])
        j_next = j + step
        if (0 <=  j_next < n): #in our domain
          if T[i,j_next] == -2:
            T[i,j_next] = T[i,j]+1
            bank_temp.append([i,j_next])
            
      if T[origin[0],origin[1]] != -2:
        print('min time is found and it is ',T[origin[0],origin[1]])
        return T
    bools = check(T,m,n)        
    bank = bank_temp
    if bank == []:
      print('some of the states have infinity minimum time, represented by -2')
      raise Exception('origin cannot reach destination at finite time')
      return T
        
        
  return T


def check(T,m,n): #actually is redundant since before checking all cells the 
  #function will actually return the result already (constructed the min time 
  #matrix, but good for debugging)
  '''Function that check if there is unchecked cell in the matrix'''
  for i in range(m):
    for j in range(n):
      if T[i,j] == -2:
        return True
  return False


np.random.seed(1000)

start_time = time.time()
x = np.random.randint(-100,100,size=(20000,2))
print('random obstacles coordinates: ',x,'\n')
x = list(x)
T,U,t = mintime2d([62,89],[-59,-52],x)
print('min time matrix: ',T, '\n')
print('optimum path: ',U,'\n')
print('min time from origin to destination: ',t,'\n')
end_time = time.time()
print('total time used: ',end_time-start_time)

random obstacles coordinates:  [[ 79 -13]
 [-29  92]
 [ -6  -8]
 ...
 [-29 -66]
 [ 11  52]
 [-78 -54]] 

min time is found and it is  404
min time matrix:  [[230 231 232 ...  -2  -2  -2]
 [229 230 231 ...  -1  -2  -2]
 [228 229 230 ...  -2  -1  -2]
 ...
 [ 98  99  -1 ...  -2  -1  -1]
 [ 99 100  -1 ...  -2  -1  -1]
 [100 101 102 ...  -1  -2  -1]] 

optimum path:  [[62, 89], [62, 90], [62, 91], [62, 92], [61, 92], [60, 92], [60, 93], [60, 94], [59, 94], [59, 95], [59, 96], [59, 97], [59, 98], [59, 99], [59, 100], [58, 100], [57, 100], [56, 100], [55, 100], [54, 100], [53, 100], [52, 100], [51, 100], [50, 100], [49, 100], [48, 100], [47, 100], [46, 100], [45, 100], [44, 100], [43, 100], [42, 100], [41, 100], [40, 100], [39, 100], [38, 100], [37, 100], [36, 100], [35, 100], [34, 100], [33, 100], [32, 100], [31, 100], [30, 100], [29, 100], [28, 100], [27, 100], [26, 100], [25, 100], [24, 100], [23, 100], [22, 100], [21, 100], [20, 100], [19, 100], [18, 100], [17, 100], [16, 100], [15, 100],

In [0]:
import numpy as np
x = np.random.randint(-100,100,size=(10,10,10))
y = x[1,1,1]
print(y)
 


-26
